<h1>MNIST GAN</h1>

Author: Vaasudevan Srinivasan <br>
Created on: July 16, 2021

[nbviewer](https://nbviewer.jupyter.org/github/PacktPublishing/Mastering-Keras/blob/master/Module%203.2_%20GANs.ipynb)

In [ ]:
from tensorflow.keras.layers import Input, Dense, Conv2D, Flatten, Dropout, LeakyReLU, Reshape, Conv2DTranspose
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from tqdm.keras import TqdmCallback
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np

tf.__version__

'2.5.0'

In [ ]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0
print(train_images.shape, test_images.shape)

(60000, 28, 28) (10000, 28, 28)


In [ ]:
# select real samples
def select_real_images(dataset, n_samples):
    # choose random instances
    indices = np.random.randint(0, dataset.shape[0], n_samples)
    # retrieve selected images
    X = dataset[indices]
    # generate 'real' class labels (1)
    y = np.ones((n_samples, 1))
    return X, y

In [ ]:
def get_discriminator(in_shape=(28, 28, 1)):
  
  inputs = Input(in_shape)
  
  conv1 = Conv2D(64, (3,3), strides=(2, 2), padding='same')(inputs)
  leak1 = LeakyReLU(alpha=.2)(conv1)
  drop1 = Dropout(.4)(leak1)
  
  conv2 = Conv2D(64, (3,3), strides=(2, 2), padding='same')(drop1)
  leak2 = LeakyReLU(alpha=.2)(conv2)
  drop2 = Dropout(.4)(leak2)
  
  flat = Flatten()(drop2)
  outputs = Dense(1,activation='sigmoid')(flat)

  model = Model(inputs=inputs,outputs=outputs)
  opt = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
  model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

  return model

In [ ]:
discriminator = get_discriminator()
discriminator.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 14, 64)        640       
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 7, 7, 64)          36928     
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 7, 7, 64)          0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 7, 7, 64)          0   

In [ ]:
def get_generator(noise_dim=100):

    inputs=Input((noise_dim, ))
    
    n = 128 * 7 * 7
    dense = Dense(n)(inputs)
    leak1 = LeakyReLU(alpha=0.2)(dense)
    reshape = Reshape((7, 7, 128))(leak1)
    
    conv_tran1 = Conv2DTranspose(128, (4,4), strides=(2, 2), padding='same')(reshape)
    leak2=LeakyReLU(alpha=0.2)(conv_tran1)
    
    conv_tran2 = Conv2DTranspose(128, (4,4), strides=(2, 2), padding='same')(leak2)
    leak3 = LeakyReLU(alpha=0.2)(conv_tran2)

    outputs = Conv2D(1, (7, 7), activation='sigmoid', padding='same')(leak3)
    model = Model(inputs=inputs,outputs=outputs)

    return model

In [ ]:
noise_dim = 100
generator = get_generator(noise_dim)
generator.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
dense_3 (Dense)              (None, 6272)              633472    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 6272)              0         
_________________________________________________________________
reshape (Reshape)            (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 28, 28, 128)       2622